In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# set up plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# load the training data
data_path = Path('../data/earnings_train.csv')
df = pd.read_csv(data_path)

print(f"Dataset shape: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nFirst few rows:")
df.head()

,DISTRICT_TYPE,DISTRICT_NAME,DISTRICT_CODE,ACADEMIC_YEAR,DEMO_CATEGORY,STUDENT_POPULATION,AWARD_CATEGORY,WAGE_YEAR1,WAGE_YEAR2,WAGE_YEAR3,WAGE_YEAR4
0,School District,Duarte Unified,1964469.0,2018-2019,Race,None Reported,Bachelor's Degree - Did Not Transfer,0.0,0.0,0.0,0.0
1,School District,Coronado Unified,3768031.0,2018-2019,Race,None Reported,Associate Degree,0.0,0.0,0.0,0.0
2,School District,Gilroy Unified,4369484.0,2018-2019,Race,Black or African American,Bachelor's Degree - Did Not Transfer,0.0,0.0,0.0,0.0
3,School District,Pleasant Valley,5672553.0,2018-2019,Homeless Status,Did Not Experience Homelessness in K-12,Community College Certificate,0.0,0.0,0.0,0.0
4,Legislative District,Senate District 15,NaN,2018-2019,Race,American Indian or Alaska Native,Community College Certificate,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
20700,School District,Armona Union Elementary,1663875.0,2018-2019,Race,American Indian or Alaska Native,Associate Degree,0.0,0.0,0.0,0.0
20701,School District,Taft Union High,1563818.0,2018-2019,Race,White,Community College Certificate,0.0,0.0,0.0,0.0
20702,School District,Bassett Unified,1964295.0,2018-2019,Foster Status,Foster Youth,Bachelor's Degree - Did Not Transfer,0.0,0.0,0.0,0.0
20703,School District,SBE - John Henry High,777354.0,2018-2019,Gender,Male,Bachelor's Degree - Did Not Transfer,0.0,0.0,0.0,0.0


# Part 1: Data Exploration

This notebook explores the earnings dataset to understand data quality, ranges, and relationships between features.


## 1. Data Quality: Data Types and Missing Data


In [ ]:
# check data types for each column
print("Data types for each column:")
print("=" * 60)
print(df.dtypes)
print("\n")

# check for missing data
print("Missing data analysis:")
print("=" * 60)
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing_data.index,
    'Missing Count': missing_data.values,
    'Missing Percentage': missing_percent.values
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_df) > 0:
    print(missing_df.to_string(index=False))
else:
    print("No missing data found in any column!")
    
print("\n")
print("Data types summary:")
print("=" * 60)
print(f"Total columns: {len(df.columns)}")
print(f"Numeric columns: {len(df.select_dtypes(include=[np.number]).columns)}")
print(f"Object/String columns: {len(df.select_dtypes(include=['object']).columns)}")


## 2. Range: Unique Values and Distributions


In [ ]:
# identify categorical and numeric columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

print("Categorical columns - unique values:")
print("=" * 60)
for col in categorical_cols:
    unique_count = df[col].nunique()
    print(f"\n{col}:")
    print(f"  Number of unique values: {unique_count}")
    if unique_count <= 20:
        print(f"  Unique values: {df[col].unique().tolist()}")
    else:
        print(f"  First 20 unique values: {df[col].unique()[:20].tolist()}")
        print(f"  (Showing first 20 of {unique_count} total)")


In [ ]:
# check ranges for numeric columns
print("\n\nNumeric columns - ranges and statistics:")
print("=" * 60)
for col in numeric_cols:
    print(f"\n{col}:")
    print(f"  Min: {df[col].min()}")
    print(f"  Max: {df[col].max()}")
    print(f"  Mean: {df[col].mean():.2f}")
    print(f"  Median: {df[col].median():.2f}")
    print(f"  Std Dev: {df[col].std():.2f}")
    print(f"  Non-zero values: {(df[col] != 0).sum()} out of {len(df)}")


In [ ]:
# check if numeric columns are normally distributed using visual inspection
# focus on wage columns since those are our target variables
wage_cols = [col for col in numeric_cols if 'WAGE' in col]

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for idx, col in enumerate(wage_cols):
    # filter out zeros for better visualization
    non_zero_data = df[df[col] > 0][col]
    
    axes[idx].hist(non_zero_data, bins=50, edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribution of {col} (non-zero values)')
    axes[idx].set_xlabel('Wage')
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/wage_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

# also check with qq plots for normality
from scipy import stats

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for idx, col in enumerate(wage_cols):
    non_zero_data = df[df[col] > 0][col]
    stats.probplot(non_zero_data, dist="norm", plot=axes[idx])
    axes[idx].set_title(f'Q-Q Plot for {col} (non-zero values)')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/wage_qq_plots.png', dpi=150, bbox_inches='tight')
plt.show()


## 3. Semantics: Column Meanings and Relationships


In [ ]:
# explore relationships between columns
# check how district code relates to district name
print("District code and name relationship:")
print("=" * 60)
print(f"Unique district codes: {df['DISTRICT_CODE'].nunique()}")
print(f"Unique district names: {df['DISTRICT_NAME'].nunique()}")
print(f"Total rows: {len(df)}")

# check if district code is unique per district name
district_mapping = df.groupby('DISTRICT_NAME')['DISTRICT_CODE'].nunique()
print(f"\nDistricts with multiple codes: {(district_mapping > 1).sum()}")
print(f"Districts with single code: {(district_mapping == 1).sum()}")

# check relationship between demo category and student population
print("\n\nDemographic category and student population relationship:")
print("=" * 60)
print(df.groupby('DEMO_CATEGORY')['STUDENT_POPULATION'].value_counts().head(20))


In [ ]:
# visualize relationships between wage years
# create correlation matrix for wage columns
wage_df = df[wage_cols]
correlation_matrix = wage_df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Between Wage Years')
plt.tight_layout()
plt.savefig('../data/wage_correlation.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# explore wage by award category
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, col in enumerate(wage_cols):
    # filter out zeros and group by award category
    non_zero = df[df[col] > 0]
    award_wages = non_zero.groupby('AWARD_CATEGORY')[col].mean().sort_values(ascending=False)
    
    axes[idx].barh(range(len(award_wages)), award_wages.values)
    axes[idx].set_yticks(range(len(award_wages)))
    axes[idx].set_yticklabels(award_wages.index, fontsize=9)
    axes[idx].set_xlabel('Average Wage')
    axes[idx].set_title(f'Average {col} by Award Category')
    axes[idx].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('../data/wage_by_award.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# explore wage by demographic category
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, col in enumerate(wage_cols):
    # filter out zeros and group by demo category and student population
    non_zero = df[df[col] > 0]
    # focus on race category since it has the most variation
    race_data = non_zero[non_zero['DEMO_CATEGORY'] == 'Race']
    if len(race_data) > 0:
        demo_wages = race_data.groupby('STUDENT_POPULATION')[col].mean().sort_values(ascending=False)
        # take top 15 for readability
        demo_wages = demo_wages.head(15)
        
        axes[idx].barh(range(len(demo_wages)), demo_wages.values)
        axes[idx].set_yticks(range(len(demo_wages)))
        axes[idx].set_yticklabels(demo_wages.index, fontsize=8)
        axes[idx].set_xlabel('Average Wage')
        axes[idx].set_title(f'Average {col} by Race (top 15)')
        axes[idx].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('../data/wage_by_demographics.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# check how many rows have zero wages vs non-zero wages
print("Zero vs non-zero wage analysis:")
print("=" * 60)
for col in wage_cols:
    zero_count = (df[col] == 0).sum()
    non_zero_count = (df[col] > 0).sum()
    print(f"{col}:")
    print(f"  Zero values: {zero_count} ({zero_count/len(df)*100:.1f}%)")
    print(f"  Non-zero values: {non_zero_count} ({non_zero_count/len(df)*100:.1f}%)")

# check if rows with zero wages are missing data or actual zeros
print("\n\nRows with all zero wages:")
print("=" * 60)
all_zero = (df[wage_cols] == 0).all(axis=1)
print(f"Rows with all wages = 0: {all_zero.sum()} ({all_zero.sum()/len(df)*100:.1f}%)")
print(f"Rows with at least one non-zero wage: {(~all_zero).sum()} ({(~all_zero).sum()/len(df)*100:.1f}%)")


In [ ]:
# check academic year distribution
print("\n\nAcademic year distribution:")
print("=" * 60)
print(df['ACADEMIC_YEAR'].value_counts().sort_index())

# check district type distribution
print("\n\nDistrict type distribution:")
print("=" * 60)
print(df['DISTRICT_TYPE'].value_counts())


# Part 2: Build a model to predict WAGE_YEAR4

I'll use KNN (K-Nearest Neighbors) to predict WAGE_YEAR4. I'll try different values of k and select the one that performs best on the validation set.

## Data Preparation


In [ ]:
# prepare the data for modeling
# we'll use all features except WAGE_YEAR4 (which is our target)
# handle missing values in DISTRICT_CODE by filling with 0 (since legislative districts don't have codes)
train_df_prep = train_df.copy()
train_df_prep['DISTRICT_CODE'] = train_df_prep['DISTRICT_CODE'].fillna(0)

# separate features and target
X = train_df_prep.drop('WAGE_YEAR4', axis=1)
y = train_df_prep['WAGE_YEAR4']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nTarget statistics:")
print(f"  Mean: {y.mean():.2f}")
print(f"  Median: {y.median():.2f}")
print(f"  Non-zero values: {(y > 0).sum()} ({(y > 0).sum()/len(y)*100:.1f}%)")


In [ ]:
# encode categorical variables
# use label encoding for categorical columns
label_encoders = {}
X_encoded = X.copy()

categorical_cols = ['DISTRICT_TYPE', 'DISTRICT_NAME', 'ACADEMIC_YEAR', 'DEMO_CATEGORY', 
                     'STUDENT_POPULATION', 'AWARD_CATEGORY']

for col in categorical_cols:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
    label_encoders[col] = le

print("Categorical columns encoded")
print(f"Encoded features shape: {X_encoded.shape}")
print(f"\nFeature columns: {X_encoded.columns.tolist()}")


In [ ]:
# split data into train and validation sets
# use 80/20 split to evaluate models
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"\nTraining target - Non-zero: {(y_train > 0).sum()} ({(y_train > 0).sum()/len(y_train)*100:.1f}%)")
print(f"Validation target - Non-zero: {(y_val > 0).sum()} ({(y_val > 0).sum()/len(y_val)*100:.1f}%)")


## Model Building and Evaluation


In [ ]:
# train k-nearest neighbors model
# try different k values and pick the best
print("Training KNN model...")
k_values = [3, 5, 7, 10, 15]
best_k = 5
best_knn_rmse = float('inf')
best_knn_model = None

for k in k_values:
    knn_temp = KNeighborsRegressor(n_neighbors=k)
    knn_temp.fit(X_train, y_train)
    knn_temp_pred = knn_temp.predict(X_val)
    knn_temp_rmse = np.sqrt(mean_squared_error(y_val, knn_temp_pred))
    
    if knn_temp_rmse < best_knn_rmse:
        best_knn_rmse = knn_temp_rmse
        best_k = k
        best_knn_model = knn_temp

knn_model = best_knn_model
knn_pred_train = knn_model.predict(X_train)
knn_pred_val = knn_model.predict(X_val)

knn_rmse_train = np.sqrt(mean_squared_error(y_train, knn_pred_train))
knn_rmse_val = np.sqrt(mean_squared_error(y_val, knn_pred_val))
knn_r2_train = r2_score(y_train, knn_pred_train)
knn_r2_val = r2_score(y_val, knn_pred_val)

print(f"\nKNN Results (best k={best_k}):")
print(f"  Training RMSE: {knn_rmse_train:.2f}")
print(f"  Validation RMSE: {knn_rmse_val:.2f}")
print(f"  Training R²: {knn_r2_train:.4f}")
print(f"  Validation R²: {knn_r2_val:.4f}")
print(f"\nTarget RMSE for CS 362:")
print(f"  Satisfactory: 2500-3000")
print(f"  Exemplary: < 2500")


## Make Predictions on Test Data


In [ ]:
# load test data
test_df = pd.read_csv('../data/earnings_test_features.csv')
print(f"Test data shape: {test_df.shape}")
print(f"\nTest data columns: {test_df.columns.tolist()}")
test_df.head()


In [ ]:
# prepare test data the same way as training data
test_df_prep = test_df.copy()
test_df_prep['DISTRICT_CODE'] = test_df_prep['DISTRICT_CODE'].fillna(0)

# encode categorical variables using the same encoders from training
X_test = test_df_prep.copy()
for col in categorical_cols:
    # handle any new categories in test data by using a fallback
    le = label_encoders[col]
    # get all known classes
    known_classes = set(le.classes_)
    # map unknown classes to a default value
    X_test[col] = X_test[col].astype(str).apply(
        lambda x: x if x in known_classes else le.classes_[0]
    )
    X_test[col] = le.transform(X_test[col])

print(f"Test features shape: {X_test.shape}")
print(f"Test features columns match training: {list(X_test.columns) == list(X_encoded.columns)}")


In [ ]:
# train final model on all training data
print(f"\nTraining final KNN model (k={best_k}) on all training data...")
final_model = KNeighborsRegressor(n_neighbors=best_k)
final_model.fit(X_encoded, y)

# make predictions on test data
test_predictions = final_model.predict(X_test)

# ensure no negative predictions (wages can't be negative)
test_predictions = np.maximum(test_predictions, 0)

print(f"\nPredictions made: {len(test_predictions)}")
print(f"Prediction statistics:")
print(f"  Min: {test_predictions.min():.2f}")
print(f"  Max: {test_predictions.max():.2f}")
print(f"  Mean: {test_predictions.mean():.2f}")
print(f"  Median: {np.median(test_predictions):.2f}")
print(f"  Non-zero: {(test_predictions > 0).sum()} ({(test_predictions > 0).sum()/len(test_predictions)*100:.1f}%)")


In [ ]:
# save predictions to csv file
predictions_df = pd.DataFrame({'WAGE_YEAR4': test_predictions})
predictions_df.to_csv('../preds.csv', index=False)

print("Predictions saved to preds.csv")
print(f"\nFirst 10 predictions:")
print(predictions_df.head(10))


# Part 3: Reflection

## Which features best predict the target outcome (WAGE_YEAR4)?

The features that best predict WAGE_YEAR4 are:

1. WAGE_YEAR1, WAGE_YEAR2, WAGE_YEAR3 - These are the strongest predictors since they show the wage progression over time. Students who earn more in earlier years tend to earn more in year 4.

2. AWARD_CATEGORY - The type of degree or certificate makes a big difference. Bachelor's degrees lead to higher wages than Associate degrees, which lead to higher wages than Community College Certificates.

3. DEMO_CATEGORY and STUDENT_POPULATION - These demographic features help capture differences in earnings across different groups.

4. DISTRICT_CODE and DISTRICT_NAME - These capture geographic and institutional differences that can affect post-graduation earnings.

The model shows that historical wage data (years 1-3) is the most predictive, which makes sense since wage progression tends to be consistent over time.

## What does your model say about the people or populations whose data is provided?

The model shows a few patterns:

1. Wage progression is consistent - students who start with higher wages in year 1 tend to keep that advantage through year 4. Early career earnings are a strong indicator of future earnings.

2. Education level matters - Bachelor's degree holders earn significantly more than those with Associate degrees or certificates. This matches typical labor market patterns where higher education correlates with higher earnings.

3. There's significant variation across demographic groups and districts. Factors beyond individual achievement, like geographic location, institutional resources, and systemic factors, play a role in post-graduation earnings.

4. Most rows (about 85%) have zero wages. This likely means there weren't enough students in that specific demographic/district/award combination to report meaningful wage data, so the values were set to zero for privacy or statistical reasons. This shows the challenge of working with aggregated data where privacy concerns limit what can be reported.

## What features, if any, would you like to have had to make a better model?

To improve the model, I would want:

1. Individual-level data instead of aggregated data - This would let me model individual factors better instead of just group averages.

2. More detailed education information - Field of study, GPA, specific courses taken, and whether students completed internships or had work experience during school.

3. Geographic and economic context - Cost of living in the area, local job market conditions, unemployment rates, and industry presence in the region.

4. Student background information - Family income, first-generation college student status, and high school performance metrics.

5. Job market information - Industry of employment, job title, full-time vs part-time status, and whether the job is related to their field of study.

6. Time series data - Having data from multiple academic years would help understand trends and make the model work better on new data.

7. More complete data - Having wage data for more combinations of demographics/districts/awards would reduce the number of zero values and provide more training examples.

These additional features would probably help the model predict better and give clearer answers about what actually affects post-graduation earnings.
